In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import plotly.subplots as sp
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import plotly.express as px
import seaborn as sns
from pandas.api.types import CategoricalDtype
import matplotlib.font_manager as fm
from matplotlib.ft2font import FT2Font
import os, warnings, math
from IPython.display import display, HTML

# Disable warnings and set display options; set plotly renderer to 'notebook'
warnings.filterwarnings("ignore")
pd.options.display.width = 5000
pd.set_option('display.max_rows', 999)
pio.renderers.default = 'notebook' #'iframe' #


c:\Users\adams\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
# title = 'THR02A-AAT-SW-P2'
title = 'SEN05-ALB_ELISA'
project='SEN04'
analyte = 'ALB'
filename = 'SEN04_20230831-ALB ELISA.xlsx'
path = os.path.join('input', project, filename)

path = r"G:\My Drive\1. Work\1.1 Projects\SEN\SEN05-20230831-ALB ELISA-raw data.xlsx"

# Constants
DILUTION_FACTOR = 50
N_STD_CURVES = 2
VOLUME = 100
CELL_NO = 55e3
DURATION = 48 #hours

# Standard curve concentrations
std_curve_concs = {
    'AAT': [1000, 200, 40, 8, 1.6, 0.32, 0.064, 0],
    'ALB': [400, 200, 100, 50, 25, 12.5, 6.25, 0],
    'mAST': [10000,5000,2500,1250,625,312.5,156.25,0]
}
std_curve_concs = std_curve_concs[analyte]


In [18]:
# Load data
x = np.array(std_curve_concs)
data = pd.read_excel(
    path,
    sheet_name='Microplate End point',
    index_col=[0]
)
data = data.loc['A':,:]
data.columns = list(range(1,13))
data = data.apply(pd.to_numeric, errors='coerce')
layout = pd.read_excel(path, sheet_name='Layout', index_col=[0])

# Extract standard values and sample names
std_curves = []
for n in range(N_STD_CURVES):
    std_curves.append(data.iloc[:, n])
    
# std1, std2 = data.iloc[:, 0], data.iloc[:, 1]
y_samples = data.iloc[:, N_STD_CURVES+1:].values.flatten()
sample_names = layout.iloc[:, N_STD_CURVES+1:].values.flatten()

# Display tables using Pandas formatting
display(HTML("<b>Raw absorbance values:</b>"),data)
display(HTML("<b>Plate layout:</b>"),layout)


,1,2,3,4,5,6,7,8,9,10,11,12
A,2.12627,2.01351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,2.01783,1.93538,1.64674,1.70206,1.59858,1.46215,1.48579,NaN,NaN,NaN,NaN,NaN
C,1.78834,1.70170,1.66892,1.67511,1.15743,1.59354,1.44160,NaN,NaN,NaN,NaN,NaN
D,1.47395,1.44828,0.72171,1.70833,1.78143,1.41955,1.16638,NaN,NaN,NaN,NaN,NaN
E,1.16868,1.14526,1.79205,1.81249,1.73070,1.66295,1.53763,NaN,NaN,NaN,NaN,NaN
F,0.97241,0.92394,1.97783,1.81754,1.85777,1.60504,1.58161,NaN,NaN,NaN,NaN,NaN
G,0.84002,0.84557,1.95653,1.77865,1.84505,1.65411,1.65346,NaN,NaN,NaN,NaN,NaN
H,0.81358,0.76147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1,2,3,4,5,6,7,8,9,10,11,12
A,400.00,400.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,200.00,200.00,Control,Doxocyclin 0.2uM,Doxocyclin 0.5uM,Palbociclib 1.5uM,Palbociclib 2uM,NaN,NaN,NaN,NaN,NaN
C,100.00,100.00,Control,Doxocyclin 0.2uM,Doxocyclin 0.5uM,Palbociclib 1.5uM,Palbociclib 2uM,NaN,NaN,NaN,NaN,NaN
D,50.00,50.00,Control,Doxocyclin 0.2uM,Doxocyclin 0.5uM,Palbociclib 1.5uM,Palbociclib 2uM,NaN,NaN,NaN,NaN,NaN
E,25.00,25.00,Control,Doxocyclin 0.2uM,Doxocyclin 0.5uM,Palbociclib 1.5uM,Palbociclib 2uM,NaN,NaN,NaN,NaN,NaN
F,12.50,12.50,Control,Doxocyclin 0.2uM,Doxocyclin 0.5uM,Palbociclib 1.5uM,Palbociclib 2uM,NaN,NaN,NaN,NaN,NaN
G,6.25,6.25,Control,Doxocyclin 0.2uM,Doxocyclin 0.5uM,Palbociclib 1.5uM,Palbociclib 2uM,NaN,NaN,NaN,NaN,NaN
H,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
def logistic4(x, A, B, C, D):
    """4PL logistic equation."""
    return ((A-D)/(1.0+((x/C)**B))) + D

def logistic4_x(y, A, B, C, D):
    """Inverse 4PL logistic equation."""
    return C * ((A-D)/(y-D) - 1)**(1/B)

def residuals(p, y, x):
    """Deviations of data from fitted 4PL curve."""
    return y - logistic4(x, *p)

def least_square(resid, p, y, x):
    """Least squares optimization for curve fitting."""
    return leastsq(resid, p, args=(y, x))[0]

# Initial parameter guess and data preparation
A = std_curves[0].min()
B = A/2
C = (std_curves[0].max()+std_curves[0].min())/1.5
D = std_curves[0].max()*1.5
p0 = [A, B, C, D]
std_mean = np.mean(np.array(std_curves), axis=0)

# Fit 4PL curve using least squares optimization
params = least_square(residuals, p0, std_mean, x)
A, B, C, D = params

# Calculate fitted values and interpolate sample concentrations
x_fit = np.arange(0, max(std_curve_concs))
y_fit = logistic4(x_fit, A, B, C, D)
samples_interp = logistic4_x(y_samples, A, B, C, D)
interpolated_concs = logistic4_x(data, A, B, C, D)
#interpolated_concs.iloc[:, N_STD_CURVES:] *= DILUTION_FACTOR

# Calculate limits of linear range (Sebaugh & McCray, 2003)
#Sebaugh, J.L. and McCray, P.D. (2003), Defining the linear portion of a sigmoid-shaped curve: bend points. 
#Pharmaceut. Statist., 2: 167-174. https://doi.org/10.1002/pst.62
k = 4.680498579882905
limit_low = (A-D) / (1 + 1/k) + D
limit_high = (A-D) / (1 + k) + D


In [36]:
print(p0)
print(A,B,C,D)


[0.81358, 0.40679, 1.9599, 3.189405]
0.7803336059568786 1.3799277545265454 51.06842863651385 2.1473010129646943


In [37]:
def ELISA_plot(x_, y_, title, standards, fit, sample_names):
    fig = go.Figure()
    
    # Add fitted curve, standard curves, and samples to the plot
    fig.add_trace(go.Scatter(x=fit[0], y=fit[1], name='Fit', mode='lines'))
    for s in standards[0]:
        fig.add_trace(go.Scatter(x=standards[1].tolist(), y=s.tolist(), name=f'Standard curve {n+1}', mode='markers'))
        
    fig.add_trace(go.Scatter(x=x_, y=y_, name='Samples', customdata=sample_names,
                             hovertemplate='<b>%{customdata}</b><br>Conc: %{x:.2f}, Abs: %{y:.2f}',
                             mode='markers', marker_symbol='x-thin', marker_line_width=2,
                             marker_line_color='#AB63FA', marker_size=7))

    # Configure plot layout, axes, and background
    fig.update_layout(margin=dict(l=20, r=20, t=40, b=20),
                      shapes=[dict(type="rect", xref="x", yref="y", x0='0', y0=str(limit_low), x1=f'{max(x)}', y1=str(limit_high),
                                  fillcolor="limegreen", opacity=0.05, line_width=0, layer="above")],
                      yaxis=dict(titlefont=dict(size=20), tickfont=dict(size=16)),
                      xaxis=dict(titlefont=dict(size=20), tickfont=dict(size=16), tickangle=-45),
                      title=title, plot_bgcolor='rgb(255,255,255)')

    # Configure x-axis and y-axis
    title_text = {'ALB': 'Albumin concentration (ng/mL)', 'AAT': 'AAT concentration (ng/mL)', 'mAST':'mAST concentration (pg/mL)'}

    fig.update_xaxes(range=(np.log10(x[-2]), 0.5 * np.ceil(2.0 * np.log10(max(x)))), title_text=title_text[analyte], type='log', dtick=0.1,
                     showgrid=True, gridwidth=0.2, gridcolor='gainsboro', zerolinecolor='Gray', zerolinewidth=0.5)
    
    # fig.update_yaxes(title_text='Absorbance (au)', dtick=0.5, showgrid=True, gridwidth=0.5, gridcolor='gainsboro',
    #                  zerolinecolor='Gray', zerolinewidth=0.5, range=(min(A - 1, D - 1), max(A + 1, D + 1)))
    
    
    # Update tick format and add spike lines
    fig.update_layout(xaxis_tickformat='.1f', yaxis_tickformat='.1f')
    fig.update_xaxes(anchor='x2', showspikes=True, spikethickness=1)
    fig.update_yaxes(showspikes=True, spikethickness=1)

    # Add vertical and horizontal lines to the plot
    fig.add_vline(x=C, opacity=0.4, line=dict(color='green'))
    fig.add_hline(y=A, opacity=0.5, line=dict(color='red'))
    fig.add_hline(y=D, opacity=0.5, line=dict(color='red'))

    display(HTML(f'<b>Lower limit of linearity: </b> Abs: {limit_low:.2f}, Conc: {logistic4_x(limit_low, A, B, C, D):.2f}'))
    display(HTML(f'<b>Upper limit of linearity: </b> Abs: {limit_high:.2f}, Conc: {logistic4_x(limit_high, A, B, C, D):.2f}'))
    # Display the plot
    fig.show()

    #output to html
    fig.write_html(f"output/{title}.html")


In [38]:
ELISA_plot(x_=samples_interp,y_=y_samples,
           title=title+' ELISA',
           standards=[std_curves,x],
           fit=[x_fit,y_fit],
          sample_names=sample_names)


In [39]:
# Define x and y axis labels
x_heat = list(range(1, 13))
y_heat = list('ABCDEFGH')

#fill in missing values
layout = layout.fillna('')

# Create heatmap plot
fig = px.imshow(data, x=x_heat, y=y_heat, color_continuous_scale='Magma', aspect="auto",title='Absorbance measurements')

# Add text annotations to heatmap cells
fig.update_traces(text=layout, texttemplate="%{text}", hovertemplate="<b>%{y}%{x}</b><br>Sample: %{text}<br>Absorbance: %{z:.3f}")
fig.update_xaxes(side="top")
# Update x-axis and y-axis properties
fig.update_xaxes(side="top", tickmode="array", tickvals=x_heat, showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


In [40]:
fig = px.imshow(interpolated_concs, x=x_heat, y=y_heat, color_continuous_scale='Magma', aspect="auto",title='Interpolated concentrations')
if analyte == 'AAT':
    fig.update_traces(text=layout, texttemplate="%{text}", hovertemplate="<b>%{y}%{x}</b><br>Sample: %{text}<br>Concentration: %{z:,.0f} ng/mL")
elif analyte == 'ALB':
    fig.update_traces(text=layout, texttemplate="%{text}", hovertemplate="<b>%{y}%{x}</b><br>Sample: %{text}<br>Concentration: %{z:,.0f} ng/mL")
elif analyte == 'mAST':
    fig.update_traces(text=layout, texttemplate="%{text}", hovertemplate="<b>%{y}%{x}</b><br>Sample: %{text}<br>Concentration: %{z:,.0f} pg/mL")

# Update x-axis and y-axis properties
fig.update_xaxes(side="top", tickmode="array", tickvals=x_heat, showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


In [41]:
interpolated_concs.to_csv(f'output/{title}_interpolated_concentrations.csv')


In [42]:
# Calculate usable range for concentrations
usable_A = logistic4_x(limit_low, A, B, C, D)
usable_D = logistic4_x(limit_high, A, B, C, D)
print(f'Minimum concentration: {usable_A:.2f} ng/mL')
print(f'Maximum concentration: {usable_D:.2f} ng/mL')

# Filter usable_z values based on usable range
usable_z = interpolated_concs.copy()
usable_z[usable_z < usable_A] = np.nan
usable_z[usable_z > usable_D] = np.nan

# Create heatmap plot with usable_z values
fig = px.imshow(usable_z, x=x_heat, y=y_heat, color_continuous_scale='Magma', aspect="auto", title='Usable Interpolated concentrations')

# Add text annotations to heatmap cells and update hover values
fig.update_traces(text=layout, texttemplate="%{text}",
                 hovertemplate="<b>%{y}%{x}</b><br>Sample: %{text}<br>Concentration: %{customdata:,.0f} ng/mL",
                 customdata=usable_z)

fig.update_xaxes(side="top", tickmode="array", tickvals=x_heat, showgrid=False)
fig.update_yaxes(showgrid=False)

# Show plot
fig.show()


Minimum concentration: 16.69 ng/mL
Maximum concentration: 156.28 ng/mL


In [28]:
# Create a mask for values within the linear range
within_range_mask = (interpolated_concs >= usable_A) & (interpolated_concs <= usable_D)

# Create a dataframe with values within the linear range
within_range_df = interpolated_concs.where(within_range_mask)

# Create a dataframe with values outside the linear range
outside_range_df = interpolated_concs.where(~within_range_mask)

# Melt the outside_range_df dataframe to a long format
outside_range_long = outside_range_df.melt(ignore_index=False, var_name='Sample name', value_name='Interpolated conc')

# Filter out rows with NaN values
filtered_outside_range = outside_range_long.dropna()

print("Dataframe with values outside the linear range:")
filtered_outside_range


Dataframe with values outside the linear range:


,Sample name,Interpolated conc
A,1,1040.005798
B,1,262.182526
F,1,13.746053
G,1,5.454180
H,1,3.517752
A,2,255.372035
B,2,174.510689
F,2,10.812551
G,2,5.835143
F,3,210.635544


In [29]:
print(layout.iloc[:8,2:12])
print(usable_z)


        3                 4                 5                  6                7  8  9  10 11 12
A                                                                                                
B  Control  Doxocyclin 0.2uM  Doxocyclin 0.5uM  Palbociclib 1.5uM  Palbociclib 2uM               
C  Control  Doxocyclin 0.2uM  Doxocyclin 0.5uM  Palbociclib 1.5uM  Palbociclib 2uM               
D  Control  Doxocyclin 0.2uM  Doxocyclin 0.5uM  Palbociclib 1.5uM  Palbociclib 2uM               
E  Control  Doxocyclin 0.2uM  Doxocyclin 0.5uM  Palbociclib 1.5uM  Palbociclib 2uM               
F  Control  Doxocyclin 0.2uM  Doxocyclin 0.5uM  Palbociclib 1.5uM  Palbociclib 2uM               
G  Control  Doxocyclin 0.2uM  Doxocyclin 0.5uM  Palbociclib 1.5uM  Palbociclib 2uM               
H                                                                                                
           1          2           3           4           5          6          7   8   9   10  11  12
A         NaN  

In [30]:
l_layout = layout.iloc[:,2:].to_numpy().flatten()
l_usable_z = usable_z.iloc[:,2:].to_numpy().flatten()
results = pd.DataFrame({'name':l_layout,'conc':l_usable_z}).set_index('name').dropna()
results.index = results.index.astype("string")
results.sort_index(inplace=True)
results
# results = results[results.index.map(len) > 0]


,conc
name,
Control,76.000607
Control,109.027127
Control,79.990132
Doxocyclin 0.2uM,117.165250
Doxocyclin 0.2uM,115.472768
Doxocyclin 0.2uM,105.120014
Doxocyclin 0.2uM,87.852092
Doxocyclin 0.2uM,86.527661
Doxocyclin 0.2uM,81.155906


In [31]:
def ug_1e6_24h(alb_conc_ng_per_ml,VOLUME_ul,CELL_NO,DURATION_h,dil_factor=50):
    vol_ml = VOLUME_ul/1000
    ng = alb_conc_ng_per_ml*vol_ml
    ug = ng/1000
    million_cells = CELL_NO/1e6
    DURATION_24h = DURATION_h/24
    result = ug/million_cells/DURATION_24h*dil_factor
    return result

VOLUME=100
CELL_NO = 50e3
DURATION = 48
# ug_10e6_24h(alb_conc_ng_per_ml=results.conc,VOLUME_ul=VOLUME,CELL_NO=CELL_NO,DURATION_h=DURATION)

results['ug_1e6_24h']=ug_1e6_24h(alb_conc_ng_per_ml=results.conc,
                                 VOLUME_ul=VOLUME,
                                 CELL_NO=CELL_NO,
                                 DURATION_h=DURATION,
                                 dil_factor=DILUTION_FACTOR)
print(results)


                         conc  ug_1e6_24h
name                                     
Control             76.000607    3.800030
Control            109.027127    5.451356
Control             79.990132    3.999507
Doxocyclin 0.2uM   117.165250    5.858262
Doxocyclin 0.2uM   115.472768    5.773638
Doxocyclin 0.2uM   105.120014    5.256001
Doxocyclin 0.2uM    87.852092    4.392605
Doxocyclin 0.2uM    86.527661    4.326383
Doxocyclin 0.2uM    81.155906    4.057795
Doxocyclin 0.5uM    25.375928    1.268796
Doxocyclin 0.5uM   127.190033    6.359502
Doxocyclin 0.5uM   132.349520    6.617476
Doxocyclin 0.5uM   105.911450    5.295573
Doxocyclin 0.5uM    68.219302    3.410965
Doxocyclin 0.5uM    92.834650    4.641732
Palbociclib 1.5uM   67.466038    3.373302
Palbociclib 1.5uM   46.486619    2.324331
Palbociclib 1.5uM   77.294957    3.864748
Palbociclib 1.5uM   50.888190    2.544410
Palbociclib 1.5uM   78.888100    3.944405
Palbociclib 1.5uM   69.200525    3.460026
Palbociclib 2uM     65.724255    3

In [32]:
grouped = results.groupby('name').ug_1e6_24h.describe()
grouped = grouped.sort_values(by=grouped.columns[1],ascending=False)
print(grouped)

print('\nTotal sample statistical description (concs):')
print(grouped.iloc[:,1].dropna().describe())


                   count      mean       std       min       25%       50%       75%       max
name                                                                                          
Doxocyclin 0.2uM     6.0  4.944114  0.786431  4.057795  4.342938  4.824303  5.644229  5.858262
Doxocyclin 0.5uM     6.0  4.599007  2.008100  1.268796  3.718657  4.968653  6.093519  6.617476
Control              3.0  4.416964  0.901345  3.800030  3.899768  3.999507  4.725431  5.451356
Palbociclib 1.5uM    6.0  3.251870  0.674408  2.324331  2.751633  3.416664  3.763567  3.944405
Palbociclib 2uM      6.0  2.757214  0.869839  1.299071  2.495718  2.831575  3.211634  3.858979

Total sample statistical description (concs):
count    5.000000
mean     3.993834
std      0.939155
min      2.757214
25%      3.251870
50%      4.416964
75%      4.599007
max      4.944114
Name: mean, dtype: float64


In [33]:
output = title+'.xlsx'
writer = pd.ExcelWriter(f'output/{output}',engine='xlsxwriter')   
workbook=writer.book
# worksheet_1=workbook.add_worksheet('Raw data')
worksheet_2=workbook.add_worksheet('Results - concentration')
# worksheet_3=workbook.add_worksheet('Results - ug_10e6_24h')
writer.sheets['Results - concentration'] = worksheet_2

title_format = workbook.add_format({'bold': True, 'font_color': 'red'})
worksheet_2.write(0, 0, 'Layout',title_format)
layout.iloc[:8,:12].to_excel(writer,sheet_name='Results - concentration',startrow=1, startcol=0)

worksheet_2.write(11, 0, 'Interpolated concentration (ng/mL)',title_format)
usable_z.to_excel(writer,sheet_name='Results - concentration',startrow=12 , startcol=0)   

worksheet_2.write(0,0,'Results - ug_10e6_24h',title_format)
results.to_excel(writer,sheet_name='Results - ug_10e6_24h ')

writer.save()
